# Introdcution to views

A **view** is a named query stored in the PostgreSQL database. 
* A view is defined based on one or more tables, which are known as **base tables**.
* The query that defines the view is referred to as a **defining query**.

After creating a view, you can query data from it as you would from a regular table. Behind the scenes, PostgreSQL will rewrite the query against the view and its defining query, executing it to retrieve data from the base tables.

Views do not store data except for the **materialized views**. In PostgreSQL, you can create special views called materialized views that store data physically and periodically refresh it from the base tables.

The materialized views are handy in various scenarios, providing faster data access to a remote server and serving as an effective caching mechanism.

# Advantages of views

**1) Simplifying complex queries**
* Views help simplify complex queries.
* Instead of dealing with joins, aggregations, or filtering conditions, you can query from views as if they were regular tables.
* Typically, first, you create views based on complex queries and store them in the database.
* Then, you can use simple queries based on views instead of using complex queries.

**2) Security and access control**
* Views enable fine-grained control over data access.
* You can create views that expose subsets of data in the base tables, hiding sensitive information.
* This is particularly useful when you have applications that require access to distinct portions of the data.

**3) Logical data independence**
* If your applications use views, you can freely modify the structure of the base tables.
* In other words, views enable you to create a layer of abstraction over the underlying tables.

# CREATE VIEW statement

```postgresql
CREATE VIEW view_name AS
query;
```

**Using the `CREATE VIEW` statement to create a view based on a complex query**
```postgresql
CREATE VIEW customer_info AS
    SELECT first_name, last_name, email, phone, city, postal_code, country
    FROM customer
    INNER JOIN address USING (address_id)
    INNER JOIN city USING (city_id)
    INNER JOIN country USING (country_id);

SELECT * FROM customer_info; 
```

**Creating a view based on another view**
```postgresql
CREATE VIEW customer_usa AS
    SELECT * FROM
    customer_info
    WHERE
    country = 'United States'; 

SELECT * FROM customer_usa; 
```

# Replacing a view

To change the defining query of a view, you use the `CREATE OR REPLACE VIEW` statement:
```postgresql
CREATE OR REPLACE VIEW view_name AS
query;
```

If the view already exists, the statement replaces the existing view; otherwise, it creates a new view.

**Example**:
```postgresql
CREATE OR REPLACE VIEW contact AS
    SELECT first_name, last_name, email, phone
    FROM customer
    INNER JOIN address USING (address_id); 
```

# DROP VIEW statement

**SYNTAX**
```postgresql
DROP VIEW [IF EXISTS] view_name
[CASCADE | RESTRICT]; 
```

* First, specify the name of the view in the `DROP VIEW` clause.
* Second, use `IF EXISTS` to prevent an error if the view does not exist.
    * PostgreSQL will issue a notice instead of an error when you attempt to remove a nonexistent view.
    * The `IF EXISTS` is optional.
* Third, use the `CASCADE` option to remove dependent objects along with the view, or the `RESTRICT` option to reject the removal of the view if other objects depend on the view.
    * The RESTRICT option is the default.
 
**Dropping multiple views**
```postgresql
DROP VIEW [IF EXISTS] view_name1, view_name2, ...
[CASCADE | RESTRICT];
```

**Permissions**: To execute the `DROP VIEW` statement, you need to be the owner of the view or have a `DROP` privilege on it.

**Using the DROP VIEW statement to drop a view that has dependent objects**
```postgresql
DROP VIEW film_info; 
```

PostgreSQL issued an error:
```postgresql
ERROR:  cannot drop view film_info because other objects depend on it
DETAIL:  view horror_film depends on view film_info
HINT:  Use DROP ... CASCADE to drop the dependent objects too. 
```

To drop the view `film_info`, you need to drop its dependent object first or use the `CASCADE` option like this:
```postgresql
DROP VIEW film_info
CASCADE; 
```

This statement drops the `film_info` view as well as its dependent object, which is the horror_film. 

It issued the following notice:
```postgresql
NOTICE:  drop cascades to view horror_film
```

# Updatable Views

A view can be updatable if it meets certain conditions. This means that you can insert, update, or delete data from the underlying tables via the view.

A view is updatable when it meets the following conditions:

First, the defining query of the view must have exactly one entry in the FROM clause, which can be a table or another updatable view.

Second, the defining query must not contain one of the following clauses at the top level:
* GROUP BY
* HAVING
* LIMIT
* OFFSET FETCH
* DISTINCT
* WITH
* UNION
* INTERSECT
* EXCEPT

Third, the selection list of the defining query must not contain any:
* Window functions
* Set-returning function
* Aggregate functions

An updatable view may contain both updatable and non-updatable columns. If you attempt to modify a non-updatable column, PostgreSQL will raise an error.

When you execute a modification statement, such as `INSERT`, `UPDATE`, or `DELETE` to an updatable view, PostgreSQL will convert this statement into the corresponding statement of the underlying table.

If you have a `WHERE` condition in the defining query of a view, you still can update or delete the rows that are not visible through the view. However, if you want to avoid this, you can use the `WITH CHECK OPTION` to define the view.